In [1]:
import pandas as pd
import fastparquet as fp
import urllib.request
from urllib.parse import urlparse
import os
import sys
import tqdm
import src.helper.helper as hlp
import re
import zipfile as zip

In [2]:
# Config
FILE_DIR = "./"
RAW_DATA_DIR = FILE_DIR + "../data/raw/"
INTERIM_DATA_DIR = FILE_DIR + "../data/interim/"
POSTER_DIR = RAW_DATA_DIR + "posters/"

DF_FNAME = 'df_train_v1.gzip'

# Display options
pd.set_option('display.max_colwidth', None)

In [3]:
# Helper function
def check_image_exists(df):
    l_check_res = []
    for idx, row in df.iterrows():
        pfname = re.findall("\w+.jpg", row.poster_url)
        l_check_res.append(pfname[0] in l_files)

    return pd.DataFrame(l_check_res, columns=["image_exists"])

In [4]:
# Read dataframe to check
df = pd.read_parquet(INTERIM_DATA_DIR + DF_FNAME)

In [41]:
# Load names of stored files
l_files = os.listdir(POSTER_DIR)

# Check, if referenced file is present on the computer
#df_check_res = hlp.parallelize_dataframe(df, check_image_exists)
df_check_res = check_image_exists(df)

In [42]:
df_check_res.value_counts()

image_exists
True            18383
False               4
dtype: int64

In [43]:
df_check_res

,image_exists
0,True
1,True
2,True
3,True
4,True
...,...
18382,True
18383,True
18384,True
18385,True


In [6]:
df.shape

(18387, 26)

In [44]:
#Write training posters to zip file
with zip.ZipFile(INTERIM_DATA_DIR + 'posters_train_v1.zip', 'w') as myzip:
    for idx, row in tqdm.tqdm(df.loc[df_check_res.image_exists == True].iterrows(), total=len(df)):
        pfname = re.findall("\w+.jpg", row.poster_url)
        myzip.write(POSTER_DIR + pfname[0])

 41%|████      | 7544/18387 [00:05<00:35, 308.22it/s] /home/ths/miniconda3/envs/aida/lib/python3.7/zipfile.py:1506: UserWarning: Duplicate name: '../data/raw/posters/1BbQEjbhDNSCs61UwaiY8EoPPiF.jpg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
 42%|████▏     | 7803/18387 [00:09<01:05, 162.08it/s]/home/ths/miniconda3/envs/aida/lib/python3.7/zipfile.py:1506: UserWarning: Duplicate name: '../data/raw/posters/9zC6iWBtpg9Tiv5v8zcF8nXkWuN.jpg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/home/ths/miniconda3/envs/aida/lib/python3.7/zipfile.py:1506: UserWarning: Duplicate name: '../data/raw/posters/5yIy1lRNXnadnMfLp0TaGqGf8mJ.jpg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
 84%|████████▍ | 15420/18387 [03:08<00:35, 83.22it/s]/home/ths/miniconda3/envs/aida/lib/python3.7/zipfile.py:1506: UserWarning: Duplicate name: '../data/raw/posters/e7CUI63PPIooocA8zsnVSOMIuUI.jpg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
100%|█████████▉|

In [47]:
# Write corrected training file
df.loc[df_check_res.image_exists == True].to_parquet(INTERIM_DATA_DIR + DF_FNAME, compression='gzip')